<a href="https://colab.research.google.com/github/mosmtilk222/Data-Analysis-Projects/blob/main/Houses_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import re
pd.__version__

'1.3.5'

In [9]:
from numpy.core.numeric import NaN
def to_int(x):
  if x == '':
    return NaN
  else:
    return int(x)

def price(x):
  try:
    x = x.replace('$', '')
    x = x.replace(' ', '')
    return int(x)
  except ValueError:
    return NaN

def whatever_to_sqft(x):
  try:
    if 'acres' in x:
      x = x.casefold()
      x = x.replace('acres', '')
      x = x.replace(' ', '')
      x = float(x)
      x = x * 43560
      return x

    elif 'sqft' in x:
      x = x.casefold()
      x = x.replace('sqft', '')
      x = x.replace(' ', '')
      return float(x)

    elif 'sqm' in x:
      x = x.casefold()
      x = x.replace('sqm', '')
      x = x.replace(' ', '')
      x = float(x)
      x = x * 10.764
      return x
  except ValueError:
    return NaN

def format_str(x):
  x = x.lower()
  x = x.replace('county', '')
  return x

def city(x):
  try:
    x = re.split(',', x)
    return x[0]
  except IndexError:
    return pd.NA

def state(x):
  try:
    x = re.split(',', x)
    return x[1]
  except IndexError:
    return pd.NA

In [31]:
from sqlite3 import connect

conn = connect('database.db')

pd.options.display.float_format = '{:,.1f}'.format
df = pd.read_sql('SELECT * FROM \'Houses for sale\'', conn)
df.rename(columns= {'Price': 'Price (US Dollars)',
                     'Lot size': 'Lot size (sqft)',
                     'Living area': 'Living area (sqft)'
                     },
          inplace = True
          )

df

,Address,Price (US Dollars),County,Lot size (sqft),Bedrooms,Total bathrooms,Living area (sqft)
0,"Kirkland,AZ,United States",$4600000,Yavapai County,,3,3,4374sqft
1,"Paradise Valley,AZ,United States",$3100000,Maricopa County,1.38acres,2,3,2810sqft
2,"Austin,TX,United States",$1025000,Travis,,4,4,3592sqft
3,"Chandler,AZ,United States",$949999,Maricopa,,4,4,3235sqft
4,"Orlando,FL,United States",$535000,Orange,,3,4,1893sqft
...,...,...,...,...,...,...,...
11968,"Melbourne,FL,United States",$574900,,131551.0sqft,3,3,2082sqft
11969,"Melbourne,FL,United States",$709900,,131551.0sqft,3,3,2345sqft
11970,"Edmonds,WA,United States",$770000,Snohomish,604753.0sqft,2,,
11971,"Seattle,WA,United States",$889000,King,1677.0sqft,3,,1320sqft


In [32]:
df['Price (US Dollars)'] = df['Price (US Dollars)'].apply(lambda x: price(x))
df['County'] = df['County'].apply(lambda x: format_str(x))
df['Bedrooms'] = df['Bedrooms'].apply(lambda x: to_int(x))
df['Lot size (sqft)'] = df['Lot size (sqft)'].apply(lambda x: whatever_to_sqft(x))
df['Total bathrooms'] = df['Total bathrooms'].apply(lambda x: to_int(x))
df['Living area (sqft)'] = df['Living area (sqft)'].apply(lambda x: whatever_to_sqft(x))
df['City'] = df['Address'].apply(lambda x: city(x))
df['State'] = df['Address'].apply(lambda x: state(x))
df.drop(['Address'], axis=1, inplace=True)
df

,Price (US Dollars),County,Lot size (sqft),Bedrooms,Total bathrooms,Living area (sqft),City,State
0,"4,600,000.0",yavapai,NaN,3.0,3.0,"4,374.0",Kirkland,AZ
1,"3,100,000.0",maricopa,"60,112.8",2.0,3.0,"2,810.0",Paradise Valley,AZ
2,"1,025,000.0",travis,NaN,4.0,4.0,"3,592.0",Austin,TX
3,"949,999.0",maricopa,NaN,4.0,4.0,"3,235.0",Chandler,AZ
4,"535,000.0",orange,NaN,3.0,4.0,"1,893.0",Orlando,FL
...,...,...,...,...,...,...,...,...
11968,"574,900.0",,"131,551.0",3.0,3.0,"2,082.0",Melbourne,FL
11969,"709,900.0",,"131,551.0",3.0,3.0,"2,345.0",Melbourne,FL
11970,"770,000.0",snohomish,"604,753.0",2.0,NaN,NaN,Edmonds,WA
11971,"889,000.0",king,"1,677.0",3.0,NaN,"1,320.0",Seattle,WA


In [20]:
df.dtypes

Price (US Dollars)    float64
County                 object
Lot size (sqft)       float64
Bedrooms              float64
Total bathrooms       float64
Living area (sqft)    float64
City                   object
State                  object
dtype: object

In [19]:
df.convert_dtypes().dtypes

Price (US Dollars)      Int64
County                 string
Lot size (sqft)       Float64
Bedrooms                Int64
Total bathrooms         Int64
Living area (sqft)    Float64
City                   string
State                  string
dtype: object

In [13]:
df.shape

(11973, 8)

In [14]:
df.isnull()

,Price (US Dollars),County,Lot size (sqft),Bedrooms,Total bathrooms,Living area (sqft),City,State
0,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False
3,False,False,True,False,False,False,False,False
4,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...
11968,False,False,False,False,False,False,False,False
11969,False,False,False,False,False,False,False,False
11970,False,False,False,False,True,True,False,False
11971,False,False,False,False,True,False,False,False


In [15]:
df.describe()

,Price (US Dollars),Lot size (sqft),Bedrooms,Total bathrooms,Living area (sqft)
count,"11,956.0","3,886.0","11,435.0","10,652.0","8,960.0"
mean,"2,185,867.6","19,251,466.8",4.0,3.9,"11,615.2"
std,"3,588,716.7","388,038,958.2",1.6,2.2,"642,807.4"
min,"469,000.0",1.0,1.0,1.0,1.0
25%,"765,000.0","9,247.5",3.0,3.0,"2,082.0"
50%,"1,180,000.0","55,103.3",4.0,3.0,"2,959.0"
75%,"2,199,000.0","217,800.0",5.0,5.0,"4,336.2"
max,"89,750,000.0","14,344,917,840.0",53.0,49.0,"60,155,509.0"


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11973 entries, 0 to 11972
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Price (US Dollars)  11956 non-null  float64
 1   County              11973 non-null  object 
 2   Lot size (sqft)     3886 non-null   float64
 3   Bedrooms            11435 non-null  float64
 4   Total bathrooms     10652 non-null  float64
 5   Living area (sqft)  8960 non-null   float64
 6   City                11973 non-null  object 
 7   State               11874 non-null  object 
dtypes: float64(5), object(3)
memory usage: 748.4+ KB


In [17]:
df[df['Price (US Dollars)'] <= 500000]

,Price (US Dollars),County,Lot size (sqft),Bedrooms,Total bathrooms,Living area (sqft),City,State
825,"500,000.0",,"87,120.0",3.0,2.0,"2,277.0",Lagrange,NY
1051,"500,000.0",maricopa,"71,874.0",4.0,4.0,"3,020.0",Surprise,AZ
1249,"500,000.0",washtenaw,NaN,2.0,2.0,"1,056.0",Ann Arbor,MI
1326,"500,000.0",maricopa,NaN,4.0,2.0,"1,524.0",Phoenix,AZ
1366,"499,000.0",sarasota,NaN,3.0,3.0,"2,262.0",Lakewood Ranch,FL
...,...,...,...,...,...,...,...,...
11407,"499,900.0",ocean,NaN,4.0,3.0,"3,000.0",Brick,NJ
11409,"499,000.0",collier,"87,120.0",4.0,3.0,"1,964.0",Naples,FL
11591,"500,000.0",brevard,NaN,3.0,2.0,"1,624.0",Indialantic,FL
11592,"500,000.0",brevard,NaN,3.0,2.0,"1,722.0",Indialantic,FL


In [18]:
import altair as alt
from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

alt.Chart(counties).mark_geoshape().encode(
    color='rate:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['rate'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=400
)

alt.Chart(...)